In [ ]:
from transaction_history_processor import PortfolioHistory
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
from IPython.display import display, HTML

In [ ]:
if not os.path.exists('portfolio_history.json'):
    print("portfolio_history.json not found. Generating it now...")
    portfolio_history = PortfolioHistory()
    portfolio_history.process_transaction_history(
        input_file='transaction_history.json',
        save_output=True,
        output_file='portfolio_history.json'
    )
    print("portfolio_history.json has been generated.")

with open('portfolio_history.json', 'r') as f:
    data = json.load(f)
    sectors = data['sectors']
    portfolio_data = data['portfolios']

In [ ]:
def plot_stock_proportions(portfolio_data, date=None, show_values=True, main_threshold=3.0):
    if date is None:
        date = max(portfolio_data.keys())

    if date not in portfolio_data:
        print(f"No data available for date: {date}")
        return

    holdings = portfolio_data[date]['holdings']
    labels = list(holdings.keys())
    values = [data['quantity'] * data['market_price']
              for data in holdings.values()]

    total = sum(values)
    percentages = [(value / total) * 100 for value in values]

    main_holdings = list(zip(labels, values, percentages))
    main_holdings.sort(key=lambda x: x[2], reverse=True)

    other_holdings = [h for h in main_holdings if h[2] < main_threshold]
    if len(other_holdings) > 1:
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 10))
        others_value = sum(v for _, v, _ in other_holdings)
        others_percentage = sum(p for _, _, p in other_holdings)
        main_holdings = [h for h in main_holdings if h[2] >=
                         main_threshold] + [("Others", others_value, others_percentage)]
    else:
        fig, ax1 = plt.subplots(figsize=(10, 10))
        ax2 = None

    main_labels, main_values, main_percentages = zip(*main_holdings)
    wedges1, texts1, autotexts1 = ax1.pie(main_percentages, labels=main_labels, autopct='%1.1f%%',
                                          startangle=90, wedgeprops={'linewidth': 0.5, 'edgecolor': 'white'})

    if show_values:
        main_legend = [f'{label} (${value:,.2f})' for label, value in zip(
            main_labels, main_values)]
    else:
        main_legend = main_labels

    ax1.legend(wedges1, main_legend, title="Holdings",
               loc="center left", bbox_to_anchor=(1, 0, 0.5, 1))
    ax1.set_title(f'Stock Proportions by Value on {date}')

    if ax2:
        other_labels, other_values, other_percentages = zip(*other_holdings)
        wedges2, texts2, autotexts2 = ax2.pie(other_percentages, labels=other_labels, autopct='%1.1f%%',
                                              startangle=90, wedgeprops={'linewidth': 0.5, 'edgecolor': 'white'})

        if show_values:
            other_legend = [f'{label} (${value:,.2f})' for label, value in zip(
                other_labels, other_values)]
        else:
            other_legend = other_labels

        ax2.legend(wedges2, other_legend, title="Other Holdings",
                   loc="center left", bbox_to_anchor=(1, 0, 0.5, 1))
        ax2.set_title(f'Breakdown of Other Holdings on {date}')

    plt.tight_layout()
    plt.show()


plot_stock_proportions(portfolio_data, show_values=False)

In [ ]:
# plot_stock_proportions(
#     portfolio_history, date="2023-10-26")

In [ ]:
def plot_sector_proportions(portfolio_data, sectors, date=None):
    if date is None:
        date = max(portfolio_data.keys())

    if date not in portfolio_data:
        print(f"No data available for date: {date}")
        return

    holdings = portfolio_data[date]['holdings']
    sector_counts = {}
    sector_symbols = {}
    for symbol, data in holdings.items():
        sector = sectors.get(symbol, 'ETF')
        if sector not in sector_counts:
            sector_counts[sector] = 0
            sector_symbols[sector] = []
        sector_counts[sector] += data['quantity'] * data['market_price']
        sector_symbols[sector].append(symbol)

    labels = list(sector_counts.keys())
    sizes = list(sector_counts.values())
    total = sum(sizes)
    percentages = [(size / total) * 100 for size in sizes]

    fig, ax = plt.subplots(figsize=(10, 7))
    wedges, texts, autotexts = ax.pie(sizes, autopct='%1.1f%%', startangle=140,
                                      wedgeprops={'linewidth': 0.5,
                                                  'edgecolor': 'white'},
                                      textprops={'fontsize': 8})
    plt.title(f'Sector Proportions on {date}')
    plt.axis('equal')

    for i, wedge in enumerate(wedges):
        sector = labels[i]
        symbols = sector_symbols[sector]
        symbols_text = ", ".join(symbols)

        angle = (wedge.theta2 - wedge.theta1) / 2. + wedge.theta1
        x = np.cos(np.radians(angle))
        y = np.sin(np.radians(angle))

        horizontalalignment = {-1: "right", 1: "left"}[int(np.sign(x))]
        connectionstyle = f"angle,angleA=0,angleB={angle}"

        ax.annotate(symbols_text, xy=(x, y), xytext=(1.35 * np.sign(x), 1.4 * y),
                    horizontalalignment=horizontalalignment, fontsize=8,
                    arrowprops=dict(arrowstyle="-", connectionstyle=connectionstyle))

    plt.legend(wedges, labels, title="Sectors",
               loc="center left", bbox_to_anchor=(1, 0, 0.5, 1))
    plt.show()
# Call the function with the loaded data
plot_sector_proportions(portfolio_data, sectors)

In [ ]:
def analyze_portfolio(portfolio_data, date=None, sort_by='Symbol', ascending=True):
    # If no date is specified, use the latest date
    if date is None:
        date = max(portfolio_data.keys())
    elif date not in portfolio_data:
        raise ValueError(f"No data available for date: {date}")

    data = portfolio_data[date]

    # Prepare data for DataFrame
    holdings_data = []
    previous_date = (pd.to_datetime(date) - pd.Timedelta(days=1)).strftime('%Y-%m-%d')

    # Find the closest available previous date
    previous_date = next((d for d in sorted(portfolio_data.keys(), reverse=True) if d < date), None)
    previous_data = portfolio_data.get(previous_date, None)

    for symbol, info in data['holdings'].items():
        quantity = info['quantity']
        total_cost = info['total_cost']
        market_price = info['market_price']
        market_value = quantity * market_price
        unit_cost = total_cost / quantity if quantity != 0 else 0
        gain_loss = market_value - total_cost
        performance = (market_value / total_cost - 1) * 100 if total_cost != 0 else 0

        if previous_data and symbol in previous_data['holdings']:
            previous_market_price = previous_data['holdings'][symbol]['market_price']
            daily_gain = market_price - previous_market_price
            daily_gain_percentage = (daily_gain / previous_market_price) * 100 if previous_market_price != 0 else 0
        else:
            daily_gain = market_price
            daily_gain_percentage = 'New Holding'

        holdings_data.append({
            'Symbol': symbol,
            'Quantity': quantity,
            'Unit Cost': unit_cost,
            'Market Price': market_price,
            'Total Cost': total_cost,
            'Market Value': market_value,
            'Gain/Loss': gain_loss,
            'Performance': performance,
            'Daily Gain': daily_gain,
            'Daily Gain %': daily_gain_percentage
        })

    # Create DataFrame
    df = pd.DataFrame(holdings_data)

    # Calculate totals including cash
    cash = data['cash']
    total_cost_sum = df['Total Cost'].sum()
    total_value = df['Market Value'].sum() + cash
    overall_performance = (total_value / total_cost_sum - 1) * 100

    # Calculate daily gains
    if previous_data:
        previous_value = sum(
            info['quantity'] * info['market_price'] for info in previous_data['holdings'].values()
        ) + previous_data['cash']
        daily_gain = total_value - previous_value
        daily_gain_percentage = (daily_gain / previous_value) * 100 if previous_value != 0 else 0
    else:
        daily_gain = None
        daily_gain_percentage = None

    column_order = [
        'Symbol', 'Quantity', 'Market Price', 'Market Value',
        'Unit Cost', 'Total Cost', 'Gain/Loss', 'Performance',
        'Daily Gain', 'Daily Gain %'
    ]
    df = df[column_order]

    # Sort the DataFrame
    df = df.sort_values(by=sort_by, ascending=ascending)

    # Format DataFrame
    df['Quantity'] = df['Quantity'].map('{:,.2f}'.format)
    df['Unit Cost'] = df['Unit Cost'].map('${:,.2f}'.format)
    df['Market Price'] = df['Market Price'].map('${:,.2f}'.format)
    df['Total Cost'] = df['Total Cost'].map('${:,.2f}'.format)
    df['Market Value'] = df['Market Value'].map('${:,.2f}'.format)
    df['Gain/Loss'] = df['Gain/Loss'].map('${:,.2f}'.format)
    df['Performance'] = df['Performance'].map('{:,.2f}%'.format)
    df['Daily Gain'] = df['Daily Gain'].map('${:,.2f}'.format)
    df['Daily Gain %'] = df['Daily Gain %'].apply(lambda x: f'{x:.2f}%' if isinstance(x, (int, float)) else x)

    # Display the results
    display(HTML(f"<h2>Portfolio Status as of {date}</h2>"))
    display(HTML(df.to_html(index=False, classes='dataframe')))

    print(f"\nTotal Market Value: ${total_value:,.2f}")
    print(f"Total Cost: ${total_cost_sum:,.2f}")
    print(f"Cash: ${cash:,.2f}")
    print(f"Overall Performance: {overall_performance:.2f}%")
    if daily_gain is not None:
        print(f"Daily Gain: ${daily_gain:,.2f} ({daily_gain_percentage:.2f}%)")
    else:
        print("Daily Gain: Data not available for the previous day")


analyze_portfolio(portfolio_data)  # Latest date, default sorting

In [ ]:
# Example usage for a specific date:
analyze_portfolio(portfolio_data, date="2024-04-10")